In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
from torch.nn.parameter import Parameter
from torch import Tensor
from pyDOE import lhs
from function_he_v5 import *

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

device = torch.device('cuda:0' if torch.cuda.is_available() else ' cpu')
print(device)

In [ ]:
import random
import numpy as np
import torch

seed = 1
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
if deterministic:
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
rho = 1 # density
mu = 0.01 # viscosity
k_fluid = 0.1 # thermal conductivity
inlet = 1
temp = 10
temp2 = 2
k_solid = 10 
cp = 10 # specific heat capacity
thermal_diffusivity = k_fluid / (rho * cp)
k_mu = mu / rho # kinematic viscosity

density_surface = 20000 # 단위 면적당 개수
density_line =  1000
'Fourier Feature'
scale = np.linspace(3, 3, 1)
mapping_size = 128
FF = True

'''
'domain_fluid': 0, 'bc_fluid_top': 1, 'bc_fluid_bottom': 2, 'bc_fluid_left': 3, 'bc_fluid_right': 4
'''
#mapping_size * len(scale) * 2
m_layers1 = [mapping_size * len(scale) * 2, 512, 512, 512, 512, 512, 3]
m_layers2 = [mapping_size * len(scale) * 2, 512, 512, 512, 512, 512, 1]
m_layers3 = [mapping_size * len(scale) * 2, 512, 512, 512, 512, 512, 1]

In [ ]:
collocation_points = COLLOCATION_POINTS(density_surface, density_line)
print(collocation_points.shape)

In [ ]:
import scipy.interpolate
import matplotlib.patches as patches

cor_fluid1 = np.loadtxt("./CFD/he_v5/xy_cor_fluid1.txt")
cor_fluid2 = np.loadtxt("./CFD/he_v5/xy_cor_fluid2.txt")
cor_solid = np.loadtxt("./CFD/he_v5/xy_cor_solid.txt")
cor_fluid1[:, 0] -= 0.7
cor_fluid1[:, 1] -= 0.5
cor_fluid2[:, 0] -= 0.7
cor_fluid2[:, 1] -= 0.5
cor_solid[:, 0] -= 0.7
cor_solid[:, 1] -= 0.5

u_fluid1 = np.loadtxt("./CFD/he_v5/u_fluid1.txt")
v_fluid1 = np.loadtxt("./CFD/he_v5/v_fluid1.txt")
p_fluid1 = np.loadtxt("./CFD/he_v5/p_fluid1.txt")
T_fluid1 = np.loadtxt("./CFD/he_v5/T_fluid1.txt") - 273.15
u_fluid2 = np.loadtxt("./CFD/he_v5/u_fluid2.txt")
v_fluid2 = np.loadtxt("./CFD/he_v5/v_fluid2.txt")
p_fluid2 = np.loadtxt("./CFD/he_v5/p_fluid2.txt")
T_fluid2 = np.loadtxt("./CFD/he_v5/T_fluid2.txt") - 273.15
T_solid = np.loadtxt("./CFD/he_v5/T_solid.txt") - 273.15

u_fluid_max, u_fluid_min = np.max(u_fluid1), np.min(u_fluid2)
v_fluid_max, v_fluid_min = np.max(v_fluid2), np.min(v_fluid1)
p_fluid_max, p_fluid_min = np.max(p_fluid1), np.min(p_fluid2)
T_fluid_max, T_fluid_min = np.max(T_fluid1), np.min(T_fluid2)
T_solid_max, T_solid_min = np.max(T_solid), np.min(T_solid)

vel_CFD1 = np.sqrt(u_fluid1**2 + v_fluid1**2)
vel_CFD2 = np.sqrt(u_fluid2**2 + v_fluid2**2)
vel_CFD_max = np.max(vel_CFD1)
vel_CFD_min = np.min(vel_CFD2)

CFD_results_fluid1 = np.vstack([u_fluid1, v_fluid1, p_fluid1, T_fluid1]).T
CFD_results_fluid2 = np.vstack([u_fluid2, v_fluid2, p_fluid2, T_fluid2]).T
CFD_results_solid = T_solid.reshape(-1 ,1)

def interpolate_output(x, y, us, extent):
    "Interpolates irregular points onto a mesh"

    # define mesh to interpolate onto
    xyi = np.meshgrid(
        np.linspace(extent[0], extent[1], 500),
        np.linspace(extent[2], extent[3], 500),
        indexing="ij",
    )

    # linearly interpolate points onto mesh
    us = [scipy.interpolate.griddata(
        (x, y), u, tuple(xyi)
        )
        for u in us]

    return us

def plot(cor_fluid1, cor_fluid2, cor_solid):
    x1, y1 = cor_fluid1[:, 0], cor_fluid1[:, 1]
    x2, y2 = cor_fluid2[:, 0], cor_fluid2[:, 1]
    x3, y3 = cor_solid[:, 0], cor_solid[:, 1]

    T_inter1, u_inter1, v_inter1, p_inter1 = interpolate_output(x1, y1,
                                                               [T_fluid1.reshape(-1, 1), u_fluid1.reshape(-1, 1), v_fluid1.reshape(-1, 1), p_fluid1.reshape(-1, 1)],
                                                               [-0.7, 0.7, -0.55, 0.55],
    )
    
    T_inter2, u_inter2, v_inter2, p_inter2 = interpolate_output(x2, y2,
                                                               [T_fluid2.reshape(-1, 1), u_fluid2.reshape(-1, 1), v_fluid2.reshape(-1, 1), p_fluid2.reshape(-1, 1)],
                                                               [-0.7, 0.7, -0.55, 0.55],
    )
    
    T_inter3 = interpolate_output(x3, y3,
                                  [T_solid.reshape(-1, 1)],
                                  [-0.7, 0.7, -0.55, 0.55],)

    T_inter1 = T_inter1.squeeze(axis=2)
    u_inter1 = u_inter1.squeeze(axis=2)
    v_inter1 = v_inter1.squeeze(axis=2)
    p_inter1 = p_inter1.squeeze(axis=2)
    
    T_inter2 = T_inter2.squeeze(axis=2)
    u_inter2 = u_inter2.squeeze(axis=2)
    v_inter2 = v_inter2.squeeze(axis=2)
    p_inter2 = p_inter2.squeeze(axis=2)
    
    T_inter3 = T_inter3[0].squeeze(axis=2)
    
    T_inter = np.concatenate((T_inter2, T_inter3, T_inter1), -1)
    
    plt.figure(figsize=(6, 4))
    plt.subplot(2, 2, 1)
    plt.title('T - CFD')
    plt.imshow(T_inter1.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=0, vmax=10, cmap='jet')
    plt.imshow(T_inter2.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=0, vmax=10, cmap='jet')
    plt.imshow(T_inter3.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=0, vmax=10, cmap='jet')
    plt.colorbar()
    shp=patches.Rectangle((-0.7, -0.55), 0.5, 0.06, color='white')
    plt.gca().add_patch(shp)
    shp=patches.Rectangle((0.2, -0.55), 0.5, 0.06, color='white')
    plt.gca().add_patch(shp)
    shp=patches.Rectangle((-0.7, 0.5), 0.5, 0.06, color='white')
    plt.gca().add_patch(shp)
    shp=patches.Rectangle((0.2, 0.5), 0.5, 0.06, color='white')
    plt.gca().add_patch(shp)
    plt.gca().axes.xaxis.set_visible(False)
    plt.gca().axes.yaxis.set_visible(False)

    plt.subplot(2, 2, 2)
    plt.title('u - CFD')
    plt.imshow(u_inter1.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=u_fluid_min, vmax=u_fluid_max, cmap='jet')
    plt.imshow(u_inter2.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=u_fluid_min, vmax=u_fluid_max, cmap='jet')
    plt.colorbar()
    shp=patches.Rectangle((-0.7, -0.55), 0.5, 0.06, color='white')
    plt.gca().add_patch(shp)
    shp=patches.Rectangle((0.2, -0.55), 0.5, 0.06, color='white')
    plt.gca().add_patch(shp)
    shp=patches.Rectangle((-0.7, 0.5), 0.5, 0.06, color='white')
    plt.gca().add_patch(shp)
    shp=patches.Rectangle((0.2, 0.5), 0.5, 0.06, color='white')
    plt.gca().add_patch(shp)
    plt.gca().axes.xaxis.set_visible(False)
    plt.gca().axes.yaxis.set_visible(False)

    plt.subplot(2, 2, 3)
    plt.title('v - CFD')
    plt.imshow(v_inter1.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=v_fluid_min, vmax=v_fluid_max, cmap='jet')
    plt.imshow(v_inter2.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=v_fluid_min, vmax=v_fluid_max, cmap='jet')
    plt.colorbar()
    shp=patches.Rectangle((-0.7, -0.55), 0.5, 0.06, color='white')
    plt.gca().add_patch(shp)
    shp=patches.Rectangle((0.2, -0.55), 0.5, 0.06, color='white')
    plt.gca().add_patch(shp)
    shp=patches.Rectangle((-0.7, 0.5), 0.5, 0.06, color='white')
    plt.gca().add_patch(shp)
    shp=patches.Rectangle((0.2, 0.5), 0.5, 0.06, color='white')
    plt.gca().add_patch(shp)
    plt.gca().axes.xaxis.set_visible(False)
    plt.gca().axes.yaxis.set_visible(False)

    plt.subplot(2, 2, 4)
    plt.title('p - CFD')
    plt.imshow(p_inter1.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=p_fluid_min, vmax=p_fluid_max, cmap='jet')
    plt.imshow(p_inter2.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=p_fluid_min, vmax=p_fluid_max, cmap='jet')
    plt.colorbar()
    shp=patches.Rectangle((-0.7, -0.55), 0.5, 0.06, color='white')
    plt.gca().add_patch(shp)
    shp=patches.Rectangle((0.2, -0.55), 0.5, 0.06, color='white')
    plt.gca().add_patch(shp)
    shp=patches.Rectangle((-0.7, 0.5), 0.5, 0.06, color='white')
    plt.gca().add_patch(shp)
    shp=patches.Rectangle((0.2, 0.5), 0.5, 0.06, color='white')
    plt.gca().add_patch(shp)
    plt.gca().axes.xaxis.set_visible(False)
    plt.gca().axes.yaxis.set_visible(False)
    plt.tight_layout()
    # plt.savefig('./image/facc_pinn_v4_01_50/facc_pinn_v4.png'.format(epoch),format='png', dpi=100, bbox_inches='tight', pad_inches=0)
    plt.show()

plot(cor_fluid1, cor_fluid2, cor_solid)

In [ ]:
def PDE_F_F1(model1, collocation_points):
    X = collocation_points.copy()
    X = torch.tensor(X, requires_grad=True)
    
    idx_domain_fluid = np.where(X[:, 0] == 0)
    colp_domain_fluid = X[idx_domain_fluid][:, 1:].to(device)
    Domain_fluid = model1(colp_domain_fluid)
    
    u = Domain_fluid[:, 0]
    v = Domain_fluid[:, 1]
    p = Domain_fluid[:, 2]

    u_x_y = autograd.grad(u, colp_domain_fluid, torch.ones(u.shape).to(device), retain_graph = True, create_graph = True)[0]
    du_x = u_x_y[:, 0:1]
    du_y = u_x_y[:, 1:2]
    
    u_xx_xy = autograd.grad(du_x, colp_domain_fluid, torch.ones(du_x.shape).to(device), retain_graph = True, create_graph = True)[0]
    u_yx_yy = autograd.grad(du_y, colp_domain_fluid, torch.ones(du_y.shape).to(device), retain_graph = True, create_graph = True)[0]
    
    v_x_y = autograd.grad(v, colp_domain_fluid, torch.ones(v.shape).to(device), retain_graph = True, create_graph = True)[0]
    dv_x = v_x_y[:, [0]]
    dv_y = v_x_y[:, [1]]
    
    v_xx_xy = autograd.grad(dv_x, colp_domain_fluid, torch.ones(dv_x.shape).to(device), create_graph = True)[0]
    v_yx_yy = autograd.grad(dv_y, colp_domain_fluid, torch.ones(dv_y.shape).to(device), create_graph = True)[0]
    
    p_x_y = autograd.grad(p, colp_domain_fluid, torch.ones(p.shape).to(device), create_graph = True)[0]
    
    du_xx = u_xx_xy[:, [0]]
    du_yy = u_yx_yy[:, [1]]
    
    dv_xx = v_xx_xy[:, [0]]
    dv_yy = v_yx_yy[:, [1]]
    
    dp_x = p_x_y[:, [0]]
    dp_y = p_x_y[:, [1]]
    
    u = u.reshape(-1, 1)
    v = v.reshape(-1, 1)

    pde_u = rho * (u * du_x + v * du_y) + dp_x - mu * (du_xx + du_yy)
    pde_v = rho * (u * dv_x + v * dv_y) + dp_y - mu * (dv_xx + dv_yy)
    pde_cont = du_x + dv_y
    
    return [pde_u.float(), pde_v.float(),  pde_cont.float()]

def PDE_F_F2(model3, collocation_points):
    X = collocation_points.copy()
    X = torch.tensor(X, requires_grad=True)
    
    idx_domain_fluid = np.where(X[:, 0] == 1)
    colp_domain_fluid = X[idx_domain_fluid][:, 1:].to(device)
    Domain_fluid = model3(colp_domain_fluid)
    
    u = Domain_fluid[:, 0]
    v = Domain_fluid[:, 1]
    p = Domain_fluid[:, 2]

    u_x_y = autograd.grad(u, colp_domain_fluid, torch.ones(u.shape).to(device), retain_graph = True, create_graph = True)[0]
    du_x = u_x_y[:, 0:1]
    du_y = u_x_y[:, 1:2]
    
    u_xx_xy = autograd.grad(du_x, colp_domain_fluid, torch.ones(du_x.shape).to(device), retain_graph = True, create_graph = True)[0]
    u_yx_yy = autograd.grad(du_y, colp_domain_fluid, torch.ones(du_y.shape).to(device), retain_graph = True, create_graph = True)[0]
    
    v_x_y = autograd.grad(v, colp_domain_fluid, torch.ones(v.shape).to(device), retain_graph = True, create_graph = True)[0]
    dv_x = v_x_y[:, [0]]
    dv_y = v_x_y[:, [1]]
    
    v_xx_xy = autograd.grad(dv_x, colp_domain_fluid, torch.ones(dv_x.shape).to(device), create_graph = True)[0]
    v_yx_yy = autograd.grad(dv_y, colp_domain_fluid, torch.ones(dv_y.shape).to(device), create_graph = True)[0]
    
    p_x_y = autograd.grad(p, colp_domain_fluid, torch.ones(p.shape).to(device), create_graph = True)[0]
    
    du_xx = u_xx_xy[:, [0]]
    du_yy = u_yx_yy[:, [1]]
    
    dv_xx = v_xx_xy[:, [0]]
    dv_yy = v_yx_yy[:, [1]]
    
    dp_x = p_x_y[:, [0]]
    dp_y = p_x_y[:, [1]]
    
    u = u.reshape(-1, 1)
    v = v.reshape(-1, 1)

    pde_u = rho * (u * du_x + v * du_y) + dp_x - mu * (du_xx + du_yy)
    pde_v = rho * (u * dv_x + v * dv_y) + dp_y - mu * (dv_xx + dv_yy)
    pde_cont = du_x + dv_y
    
    return [pde_u.float(), pde_v.float(),  pde_cont.float()]

def PDE_H_F1(model1, model2, collocation_points):
    X = collocation_points.copy()
    X = torch.tensor(X, requires_grad=True)
    
    idx_domain_fluid = np.where(X[:, 0] == 0)
    colp_domain_fluid = X[idx_domain_fluid][:, 1:].to(device)
    
#     with torch.no_grad():
#         model1.eval()
    Domain_fluid1_F = model1(colp_domain_fluid)
    
    Domain_fluid1_H = model2(colp_domain_fluid)
    
    u = Domain_fluid1_F[:, 0:1]
    v = Domain_fluid1_F[:, 1:2]
    T = Domain_fluid1_H[:, 0:1]
    
    u_x_y = autograd.grad(u, colp_domain_fluid, torch.ones(u.shape).to(device), retain_graph = True, create_graph = True)[0]
    du_x = u_x_y[:, 0:1]
    du_y = u_x_y[:, 1:2]
    
    T_x_y = autograd.grad(T, colp_domain_fluid, torch.ones(T.shape).to(device), retain_graph=True, create_graph=True)[0]
    dT_x = T_x_y[:, 0:1]
    dT_y = T_x_y[:, 1:2]
    
    T_xx_xy = autograd.grad(dT_x, colp_domain_fluid, torch.ones(dT_x.shape).to(device), retain_graph=True, create_graph=True)[0]
    dT_xx = T_xx_xy[:, 0:1]
    
    T_yx_yy = autograd.grad(dT_y, colp_domain_fluid, torch.ones(dT_y.shape).to(device), retain_graph=True, create_graph=True)[0]
    dT_yy = T_yx_yy[:, 1:2]
    
    u = u.detach()
    v = v.detach()
    du_y = du_y.detach()
    
    pde_t = (u * dT_x + v * dT_y) - thermal_diffusivity * (dT_xx + dT_yy) #- (k_mu/cp) * du_y**2
    
    return [pde_t.float()]

def PDE_H_F2(model3, model4, collocation_points):
    X = collocation_points.copy()
    X = torch.tensor(X, requires_grad=True)
    
    idx_domain_fluid = np.where(X[:, 0] == 1)
    colp_domain_fluid = X[idx_domain_fluid][:, 1:].to(device)
    
#     with torch.no_grad():
#         model1.eval()
    Domain_fluid2_F = model3(colp_domain_fluid)
    
    Domain_fluid2_H = model4(colp_domain_fluid)
    
    u = Domain_fluid2_F[:, 0:1]
    v = Domain_fluid2_F[:, 1:2]
    T = Domain_fluid2_H[:, 0:1]
    
    u_x_y = autograd.grad(u, colp_domain_fluid, torch.ones(u.shape).to(device), retain_graph = True, create_graph = True)[0]
    du_x = u_x_y[:, 0:1]
    du_y = u_x_y[:, 1:2]
    
    T_x_y = autograd.grad(T, colp_domain_fluid, torch.ones(T.shape).to(device), retain_graph=True, create_graph=True)[0]
    dT_x = T_x_y[:, 0:1]
    dT_y = T_x_y[:, 1:2]
    
    T_xx_xy = autograd.grad(dT_x, colp_domain_fluid, torch.ones(dT_x.shape).to(device), retain_graph=True, create_graph=True)[0]
    dT_xx = T_xx_xy[:, 0:1]
    
    T_yx_yy = autograd.grad(dT_y, colp_domain_fluid, torch.ones(dT_y.shape).to(device), retain_graph=True, create_graph=True)[0]
    dT_yy = T_yx_yy[:, 1:2]
    
    u = u.detach()
    v = v.detach()
    du_y = du_y.detach()
    
    pde_t = (u * dT_x + v * dT_y) - thermal_diffusivity * (dT_xx + dT_yy) #- (k_mu/cp) * du_y**2
    
    return [pde_t.float()]

def PDE_S(model5, collocation_points):
    X = collocation_points.copy()
    X = torch.tensor(X, requires_grad=True)
    
    idx_domain_solid = np.where(X[:, 0] == 10)[0]
    
    colp_domain_solid = X[idx_domain_solid][:, 1:].to(device)
    
    Domain_solid = model5(colp_domain_solid)
    
    T = Domain_solid[:, 0:1]
                    
    T_x_y = torch.autograd.grad(T, colp_domain_solid, torch.ones(T.shape).to(device), retain_graph=True, create_graph=True)[0]
    dT_x = T_x_y[:, 0:1]
    dT_y = T_x_y[:, 1:2]
    
    T_xx_yx = torch.autograd.grad(dT_x, colp_domain_solid, torch.ones(dT_x.shape).to(device), retain_graph=True, create_graph=True)[0]
    dT_xx = T_xx_yx[:, 0:1]
    
    T_xy_yy = torch.autograd.grad(dT_y, colp_domain_solid, torch.ones(dT_y.shape).to(device), retain_graph=True, create_graph=True)[0]
    dT_yy = T_xy_yy[:, 1:2]
                    
    pde_t = k_solid * (dT_xx + dT_yy)
                    
    return [pde_t.float()]

In [ ]:
def BC_F1(model1, collocation_points, inlet):
    X = collocation_points.copy()
    X = torch.tensor(X, requires_grad=True)
    idx_bc_fluid_top = np.where(X[:, 0] == 2)[0]
    idx_bc_fluid_bottom = np.where(X[:, 0] == 3)[0]
    idx_inlet = np.where(X[:, 0] == 6)[0]
    idx_outlet = np.where(X[:, 0] == 7)[0]
    colp_bc = X[:, 1:].to(device)
    
    BC_fluid_top = model1(colp_bc[idx_bc_fluid_top])[:, 0:2]
    BC_fluid_bottom = model1(colp_bc[idx_bc_fluid_bottom])[:, 0:2]
    BC_inlet_u = model1(colp_bc[idx_inlet])[:, 0]
    BC_inlet_v = model1(colp_bc[idx_inlet])[:, 1] - torch.ones_like(model1(colp_bc[idx_inlet])[:, 1]) * (-1 * inlet)
    BC_outlet_v = model1(colp_bc[idx_outlet])[:, 1] 
    BC_outlet_p = model1(colp_bc[idx_outlet])[:, 2] 
    
    return [BC_fluid_top.float(), BC_fluid_bottom.float(), BC_inlet_u.float(), BC_inlet_v.float(), BC_outlet_v.float(), BC_outlet_p.float()]

def BC_F2(model3, collocation_points, inlet):
    X = collocation_points.copy()
    X = torch.tensor(X, requires_grad=True)
    idx_bc_fluid_top = np.where(X[:, 0] == 4)[0]
    idx_bc_fluid_bottom = np.where(X[:, 0] == 5)[0]
    idx_inlet = np.where(X[:, 0] == 8)[0]
    idx_outlet = np.where(X[:, 0] == 9)[0]
    colp_bc = X[:, 1:].to(device)
    
    BC_fluid_top = model3(colp_bc[idx_bc_fluid_top])[:, 0:2]
    BC_fluid_bottom = model3(colp_bc[idx_bc_fluid_bottom])[:, 0:2]
    BC_inlet_u = model3(colp_bc[idx_inlet])[:, 0] 
    BC_inlet_v = model3(colp_bc[idx_inlet])[:, 1] - torch.ones_like(model3(colp_bc[idx_inlet])[:, 1]) * inlet
    BC_outlet_v = model3(colp_bc[idx_outlet])[:, 1] 
    BC_outlet_p = model3(colp_bc[idx_outlet])[:, 2] 
    
    return [BC_fluid_top.float(), BC_fluid_bottom.float(), BC_inlet_u.float(), BC_inlet_v.float(), BC_outlet_v.float(), BC_outlet_p.float()]

def BC_T1(model2, collocation_points):
    X = collocation_points.copy()
    X = torch.tensor(X, requires_grad=True)
    idx_bc_fluid_top = np.where(X[:, 0] == 2)[0]
    idx_inlet_fluid = np.where(X[:, 0] == 6)[0]
    idx_outlet_fluid = np.where(X[:, 0] == 7)[0]

    colp_bc = X[:, 1:].to(device)
    colp_outlet_fluid = X[idx_outlet_fluid][:, 1:].to(device)
    
    T_fluid_top = model2(colp_bc[idx_bc_fluid_top])[:, 0]
    T_outlet = model2(colp_outlet_fluid)[:, 0]
    T_outlet_x_y = autograd.grad(T_outlet, colp_outlet_fluid, torch.ones(T_outlet.shape).to(device), retain_graph=True, create_graph=True)[0]
    dT_outlet_x = T_outlet_x_y[:, 0:1]
    
    T_inlet = model2(colp_bc[idx_inlet_fluid])[:, 0] - torch.ones_like(model2(colp_bc[idx_inlet_fluid])[:, 0]) * temp
    
    return [T_fluid_top.float(), dT_outlet_x.float(), T_inlet.float()]

def BC_T2(model4, collocation_points):
    X = collocation_points.copy()
    X = torch.tensor(X, requires_grad=True)
    idx_bc_fluid_bottom = np.where(X[:, 0] == 5)[0]
    idx_inlet_fluid = np.where(X[:, 0] == 8)[0]
    idx_outlet_fluid = np.where(X[:, 0] == 9)[0]

    colp_bc = X[:, 1:].to(device)
    colp_outlet_fluid = X[idx_outlet_fluid][:, 1:].to(device)
    
    T_fluid_bottom = model4(colp_bc[idx_bc_fluid_bottom])[:, 0]
    T_outlet = model4(colp_outlet_fluid)[:, 0]
    T_outlet_x_y = autograd.grad(T_outlet, colp_outlet_fluid, torch.ones(T_outlet.shape).to(device), retain_graph=True, create_graph=True)[0]
    dT_outlet_x = T_outlet_x_y[:, 0:1]
    
    T_inlet = model4(colp_bc[idx_inlet_fluid])[:, 0] - torch.ones_like(model4(colp_bc[idx_inlet_fluid])[:, 0]) * temp2
    
    return [T_fluid_bottom.float(), dT_outlet_x.float(), T_inlet.float()]

In [ ]:
import matplotlib.patches as patches

def PLOT(model1, model2, col_fluid1, col_fluid2, col_solid, CFD_fluid1, CFD_fluid2, CFD_solid, epoch, history=False):
    X1 = col_fluid1.copy()
    X2 = col_fluid2.copy()
    X3 = col_solid.copy()
    X1 = torch.tensor(X1, requires_grad=False)
    X2 = torch.tensor(X2, requires_grad=False)
    X3 = torch.tensor(X3, requires_grad=False)
    
    with torch.no_grad():
        model1.eval()
        model2.eval()

        plt.figure(figsize=(7.5, 6.7))
        test_result1 = model1(X1.to(device))
        test_result2 = model2(X1.to(device))
        test_result3 = model1(X2.to(device))
        test_result4 = model2(X2.to(device))
        test_result5 = model2(X3.to(device))

        u1_test = (test_result1[:, 0].detach().cpu().numpy()).reshape(-1, 1)
        v1_test = (test_result1[:, 1].detach().cpu().numpy()).reshape(-1, 1)
        p1_test = (test_result1[:, 2].detach().cpu().numpy()).reshape(-1, 1)
        T1_test = (test_result2[:, 0].detach().cpu().numpy()).reshape(-1, 1)
        
        u2_test = (test_result3[:, 0].detach().cpu().numpy()).reshape(-1, 1)
        v2_test = (test_result3[:, 1].detach().cpu().numpy()).reshape(-1, 1)
        p2_test = (test_result3[:, 2].detach().cpu().numpy()).reshape(-1, 1)
        T2_test = (test_result4[:, 0].detach().cpu().numpy()).reshape(-1, 1)
        
        T3_test = (test_result5[:, 0].detach().cpu().numpy()).reshape(-1, 1)
        
        u1_GT = CFD_fluid1[:, 0].reshape(-1, 1)
        v1_GT = CFD_fluid1[:, 1].reshape(-1, 1)
        p1_GT = CFD_fluid1[:, 2].reshape(-1, 1)
        T1_GT = CFD_fluid1[:, 3].reshape(-1, 1)
        
        u2_GT = CFD_fluid2[:, 0].reshape(-1, 1)
        v2_GT = CFD_fluid2[:, 1].reshape(-1, 1)
        p2_GT = CFD_fluid2[:, 2].reshape(-1, 1)
        T2_GT = CFD_fluid2[:, 3].reshape(-1, 1)
        T3_GT = CFD_solid[:, 0].reshape(-1, 1)
        
        x1, y1 = X1[:, 0], X1[:, 1]
        x2, y2 = X2[:, 0], X2[:, 1]
        x3, y3 = X3[:, 0], X3[:, 1]

        T1_CFD, u1_CFD, v1_CFD, p1_CFD = interpolate_output(x1, y1,
                                                           [T1_GT, u1_GT, v1_GT, p1_GT],
                                                           [-0.7, 0.7, -0.55, 0.55],
        )
        T2_CFD, u2_CFD, v2_CFD, p2_CFD = interpolate_output(x2, y2,
                                                           [T2_GT, u2_GT, v2_GT, p2_GT],
                                                           [-0.7, 0.7, -0.55, 0.55],
        )
        T3_CFD = interpolate_output(x3, y3,
                                    [T3_GT],
                                    [-0.7, 0.7, -0.55, 0.55])
        
        T1_pred, u1_pred, v1_pred, p1_pred = interpolate_output(x1, y1,
                                                               [T1_test, u1_test, v1_test, p1_test],
                                                               [-0.7, 0.7, -0.55, 0.55],
        )
        T2_pred, u2_pred, v2_pred, p2_pred = interpolate_output(x2, y2,
                                                               [T2_test, u2_test, v2_test, p2_test],
                                                               [-0.7, 0.7, -0.55, 0.55],
        )
        T3_pred = interpolate_output(x3, y3,
                                     [T3_test],
                                     [-0.7, 0.7, -0.55, 0.55])
        
        T1_error, u1_error, v1_error, p1_error = interpolate_output(x1, y1,
                                                                   [abs(T1_test - T1_GT), abs(u1_test - u1_GT), abs(v1_test - v1_GT), abs(p1_test - p1_GT)],
                                                                   [-0.7, 0.7, -0.55, 0.55],
        )
        T2_error, u2_error, v2_error, p2_error = interpolate_output(x2, y2,
                                                                   [abs(T2_test - T2_GT), abs(u2_test - u2_GT), abs(v2_test - v2_GT), abs(p2_test - p2_GT)],
                                                                   [-0.7, 0.7, -0.55, 0.55],
        )
        T3_error = interpolate_output(x3, y3,
                                     [abs(T3_test - T3_GT)],
                                     [-0.7, 0.7, -0.55, 0.55])
        
        T1_CFD = T1_CFD.squeeze(axis=2)
        u1_CFD = u1_CFD.squeeze(axis=2)
        v1_CFD = v1_CFD.squeeze(axis=2)
        p1_CFD = p1_CFD.squeeze(axis=2)
        T2_CFD = T2_CFD.squeeze(axis=2)
        u2_CFD = u2_CFD.squeeze(axis=2)
        v2_CFD = v2_CFD.squeeze(axis=2)
        p2_CFD = p2_CFD.squeeze(axis=2)
        T3_CFD = T3_CFD[0].squeeze(axis=2)
        
        T1_pred = T1_pred.squeeze(axis=2)
        u1_pred = u1_pred.squeeze(axis=2)
        v1_pred = v1_pred.squeeze(axis=2)
        p1_pred = p1_pred.squeeze(axis=2)
        T2_pred = T2_pred.squeeze(axis=2)
        u2_pred = u2_pred.squeeze(axis=2)
        v2_pred = v2_pred.squeeze(axis=2)
        p2_pred = p2_pred.squeeze(axis=2)
        T3_pred = T3_pred[0].squeeze(axis=2)
        
        T1_error = T1_error.squeeze(axis=2)
        u1_error = u1_error.squeeze(axis=2)
        v1_error = v1_error.squeeze(axis=2)
        p1_error = p1_error.squeeze(axis=2)
        T2_error = T2_error.squeeze(axis=2)
        u2_error = u2_error.squeeze(axis=2)
        v2_error = v2_error.squeeze(axis=2)
        p2_error = p2_error.squeeze(axis=2)
        T3_error = T3_error[0].squeeze(axis=2)
        
        T_CFD = np.concatenate((T2_CFD, T3_CFD, T1_CFD), -1)
        T_pred = np.concatenate((T2_pred, T3_pred, T1_pred), -1)
        T_error = np.concatenate((T2_error, T3_error, T1_error), -1)
        
        plt.subplot(4, 3, 1)
        plt.title('CFD: T')
        plt.imshow(T1_CFD.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=0, vmax=10, cmap='jet')
        plt.imshow(T2_CFD.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=0, vmax=10, cmap='jet')
        plt.imshow(T3_CFD.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=0, vmax=10, cmap='jet')
        plt.colorbar()
        shp=patches.Rectangle((-0.7, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((-0.7, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        plt.gca().axes.xaxis.set_visible(False)
        plt.gca().axes.yaxis.set_visible(False)
        
        plt.subplot(4, 3, 2)
        plt.title('MX-PINN: T')
        plt.imshow(T1_pred.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=0, vmax=10, cmap='jet')
        plt.imshow(T2_pred.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=0, vmax=10, cmap='jet')
        plt.imshow(T3_pred.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=0, vmax=10, cmap='jet')
        plt.colorbar()
        shp=patches.Rectangle((-0.7, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((-0.7, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        plt.gca().axes.xaxis.set_visible(False)
        plt.gca().axes.yaxis.set_visible(False)
        
        plt.subplot(4, 3, 3)
        plt.title('Error: T')
        plt.imshow(T1_error.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=0, vmax=2, cmap='jet')
        plt.imshow(T2_error.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=0, vmax=2, cmap='jet')
        plt.imshow(T3_error.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=0, vmax=2, cmap='jet')
        plt.colorbar()
        shp=patches.Rectangle((-0.7, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((-0.7, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        plt.gca().axes.xaxis.set_visible(False)
        plt.gca().axes.yaxis.set_visible(False)
        
        plt.subplot(4, 3, 4)
        plt.title('CFD: u')
        plt.imshow(u1_CFD.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=u_fluid_min, vmax=u_fluid_max, cmap='jet')
        plt.imshow(u2_CFD.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=u_fluid_min, vmax=u_fluid_max, cmap='jet')
        plt.colorbar()
        shp=patches.Rectangle((-0.7, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((-0.7, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        plt.gca().axes.xaxis.set_visible(False)
        plt.gca().axes.yaxis.set_visible(False)
        
        plt.subplot(4, 3, 5)
        plt.title('MX-PINN: u')
        plt.imshow(u1_pred.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=u_fluid_min, vmax=u_fluid_max, cmap='jet')
        plt.imshow(u2_pred.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=u_fluid_min, vmax=u_fluid_max, cmap='jet')
        plt.colorbar()
        shp=patches.Rectangle((-0.7, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((-0.7, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        plt.gca().axes.xaxis.set_visible(False)
        plt.gca().axes.yaxis.set_visible(False)
        
        plt.subplot(4, 3, 6)
        plt.title('Error: u')
        plt.imshow(u1_error.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=0, vmax=0.2 * u_fluid_max, cmap='jet')
        plt.imshow(u2_error.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=0, vmax=0.2 * u_fluid_max, cmap='jet')
        plt.colorbar()
        shp=patches.Rectangle((-0.7, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((-0.7, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        plt.gca().axes.xaxis.set_visible(False)
        plt.gca().axes.yaxis.set_visible(False)
               
        plt.subplot(4, 3, 7)
        plt.title('CFD: v')
        plt.imshow(v1_CFD.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=v_fluid_min, vmax=v_fluid_max, cmap='jet')
        plt.imshow(v2_CFD.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=v_fluid_min, vmax=v_fluid_max, cmap='jet')
        plt.colorbar()
        shp=patches.Rectangle((-0.7, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((-0.7, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        plt.gca().axes.xaxis.set_visible(False)
        plt.gca().axes.yaxis.set_visible(False)
                  
        plt.subplot(4, 3, 8)
        plt.title('MX-PINN: v')
        plt.imshow(v1_pred.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=v_fluid_min, vmax=v_fluid_max, cmap='jet')
        plt.imshow(v2_pred.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=v_fluid_min, vmax=v_fluid_max, cmap='jet')
        plt.colorbar()
        shp=patches.Rectangle((-0.7, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((-0.7, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        plt.gca().axes.xaxis.set_visible(False)
        plt.gca().axes.yaxis.set_visible(False)
                  
        plt.subplot(4, 3, 9)
        plt.title('Error: v')
        plt.imshow(v1_error.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=0, vmax=0.2 * v_fluid_max, cmap='jet')
        plt.imshow(v2_error.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=0, vmax=0.2 * v_fluid_max, cmap='jet')
        plt.colorbar()
        shp=patches.Rectangle((-0.7, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((-0.7, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        plt.gca().axes.xaxis.set_visible(False)
        plt.gca().axes.yaxis.set_visible(False)
        
        plt.subplot(4, 3, 10)
        plt.title('CFD: p')
        plt.imshow(p1_CFD.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=p_fluid_min, vmax=p_fluid_max, cmap='jet')
        plt.imshow(p2_CFD.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=p_fluid_min, vmax=p_fluid_max, cmap='jet')
        plt.colorbar()
        shp=patches.Rectangle((-0.7, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((-0.7, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        plt.gca().axes.xaxis.set_visible(False)
        plt.gca().axes.yaxis.set_visible(False)
                  
        plt.subplot(4, 3, 11)
        plt.title('MX-PINN: p')
        plt.imshow(p1_pred.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=p_fluid_min, vmax=p_fluid_max, cmap='jet')
        plt.imshow(p2_pred.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=p_fluid_min, vmax=p_fluid_max, cmap='jet')
        plt.colorbar()
        shp=patches.Rectangle((-0.7, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((-0.7, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        plt.gca().axes.xaxis.set_visible(False)
        plt.gca().axes.yaxis.set_visible(False)
                  
        plt.subplot(4, 3, 12)
        plt.title('Error: p')
        plt.imshow(p1_error.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=0, vmax=0.2 * p_fluid_max, cmap='jet')
        plt.imshow(p2_error.T, origin="lower", extent=[-0.7, 0.7, -0.55, 0.55], vmin=0, vmax=0.2 * p_fluid_max, cmap='jet')
        plt.colorbar()
        shp=patches.Rectangle((-0.7, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, -0.55), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((-0.7, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        shp=patches.Rectangle((0.2, 0.5), 0.5, 0.06, color='white')
        plt.gca().add_patch(shp)
        plt.gca().axes.xaxis.set_visible(False)
        plt.gca().axes.yaxis.set_visible(False)
        plt.tight_layout()
        plt.savefig('./image/he_pinn_MPINN/he_pinn_MPINN_{0:06d}.png'.format(epoch),format='png', dpi=100, bbox_inches='tight', pad_inches=0)
        plt.show()
        
        plt.close()

    if history==True:
        plt.plot(loss_record, label='total')
        plt.yscale("log")
        plt.legend()
        plt.show()
    model1.train()
    model2.train()

In [ ]:
class MODEL(nn.Module):
    def __init__(self, layers, mapping_size, scale, FF = False):
        super().__init__()
        
        self.FF = FF
        self.FourierFeature = GauossianFourierFeatureTransform_1D(2, mapping_size, scale)
        
        'activation function'
        self.activation = nn.Tanh()
        
        self.layers = layers
        'initialize neural network as a list using nn.Modulelist'
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i + 1]) for i in range(len(layers) - 1)])
        
        self.iter = 0
        
        'Xavier Normal initialization'
        for i in range(len(layers) - 1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain = 1.0) # weight from a normal distribution with recommanded gain value
            nn.init.zeros_(self.linears[i].bias.data) # set biases to zero
            
    def forward(self, x):
        if torch.is_tensor(x) != True:
            print('Input is not tensor')
        
        a = x.float()
        
        if self.FF == True:
            
            a = self.FourierFeature(a)
        
        for i in range(len(self.layers) - 2):
            z = self.linears[i](a)
            a = self.activation(z)
        
        a = self.linears[-1](a)
        
        return a

In [ ]:
'Fourier Feature'
scale = np.linspace(3, 3, 1)
mapping_size = 128
FF = True

'''
'domain_fluid': 0, 'bc_fluid_top': 1, 'bc_fluid_bottom': 2, 'bc_fluid_left': 3, 'bc_fluid_right': 4
'''
FF = True
m_layers1 = [(mapping_size * len(scale) * 2) * FF + (1 - FF) * 2, 678, 678, 678, 678, 678, 3]
model1 = MODEL(m_layers1, mapping_size, scale, FF).to(device)

FF = False
m_layers2 = [(mapping_size * len(scale) * 2) * FF + (1 - FF) * 2, 888, 888, 888, 888, 888, 1]
model2 = MODEL(m_layers2, mapping_size, scale, FF).to(device)

# FF = True
# m_layers3 = [(mapping_size * len(scale) * 2) * FF + (1 - FF) * 2, 678, 678, 678, 678, 678, 3]
# model3 = MODEL(m_layers3, mapping_size, scale, FF).to(device)

# FF = False
# m_layers4 = [(mapping_size * len(scale) * 2) * FF + (1 - FF) * 2, 888, 888, 888, 888, 888, 1]
# model4 = MODEL(m_layers4, mapping_size, scale, FF).to(device)

# FF = False
# m_layers5 = [(mapping_size * len(scale) * 2) * FF + (1 - FF) * 2, 512, 512, 512, 512, 512, 1]
# model5 = MODEL(m_layers5, mapping_size, scale, FF).to(device)

optimizer1 = torch.optim.Adam(model1.parameters(), lr=5e-5)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=5e-5)
# optimizer3 = torch.optim.Adam(model3.parameters(), lr=5e-5)
# optimizer4 = torch.optim.Adam(model4.parameters(), lr=5e-6)
# optimizer5 = torch.optim.Adam(model5.parameters(), lr=5e-6)
Loss_func = nn.MSELoss(reduction='mean')
max_loss = np.inf
max_loss1 = np.inf
max_loss2 = np.inf
max_loss3 = np.inf
max_loss4 = np.inf
max_loss5 = np.inf

In [ ]:
total_params = 0

model_list = [model1, model2]

for model in model_list:
    for name, param in model.named_parameters():
        if param.requires_grad:
            param_count = param.numel()
            total_params += param_count

# 총 파라미터 수 추가
print("Model Parameter Count {}".format(total_params))

In [ ]:
epoch = 0
Loss_pde_for = 0
Loss_bc_for = 0
cor = collocation_points[:, 1:]

loss_history = []
loss_history_pde = []
loss_history_bc = []
loss_history_sb = []
u_MSE = []
v_MSE = []
p_MSE = []
T1_MSE = []
T2_MSE = []

In [ ]:
while epoch < 150001:
    Loss_PDE_F_F1 = PDE_F_F1(model1, collocation_points)  
    Loss_BC_F1 = BC_F1(model1, collocation_points, inlet)
    Loss_PDE_F_F2 = PDE_F_F2(model1, collocation_points)  
    Loss_BC_F2 = BC_F2(model1, collocation_points, inlet)
    
    # Loss about fluid flow1
    Loss_PDE_F1_u = Loss_func(Loss_PDE_F_F1[0].float(), torch.zeros_like(Loss_PDE_F_F1[0]).to(device))
    Loss_PDE_F1_v = Loss_func(Loss_PDE_F_F1[1].float(), torch.zeros_like(Loss_PDE_F_F1[1]).to(device))
    Loss_PDE_F1_cont = Loss_func(Loss_PDE_F_F1[2].float(), torch.zeros_like(Loss_PDE_F_F1[2]).to(device)) 

    loss_pde_f1 = Loss_PDE_F1_u + Loss_PDE_F1_v + Loss_PDE_F1_cont
    
    Loss_PDE_F2_u = Loss_func(Loss_PDE_F_F2[0].float(), torch.zeros_like(Loss_PDE_F_F2[0]).to(device))
    Loss_PDE_F2_v = Loss_func(Loss_PDE_F_F2[1].float(), torch.zeros_like(Loss_PDE_F_F2[1]).to(device))
    Loss_PDE_F2_cont = Loss_func(Loss_PDE_F_F2[2].float(), torch.zeros_like(Loss_PDE_F_F2[2]).to(device)) 

    loss_pde_f2 = Loss_PDE_F2_u + Loss_PDE_F2_v + Loss_PDE_F2_cont
    
    Loss_BC_fluid_left = Loss_func(Loss_BC_F1[0].float(), torch.zeros_like(Loss_BC_F1[0]).to(device))
    Loss_BC_fluid_right = Loss_func(Loss_BC_F1[1].float(), torch.zeros_like(Loss_BC_F1[1]).to(device))
    Loss_BC_inlet_u = Loss_func(Loss_BC_F1[2].float(), torch.zeros_like(Loss_BC_F1[2]).to(device))
    Loss_BC_inlet_v = Loss_func(Loss_BC_F1[3].float(), torch.zeros_like(Loss_BC_F1[3]).to(device))
    Loss_BC_outlet_v = Loss_func(Loss_BC_F1[4].float(), torch.zeros_like(Loss_BC_F1[4]).to(device))
    Loss_BC_outlet_p = Loss_func(Loss_BC_F1[5].float(), torch.zeros_like(Loss_BC_F1[5]).to(device))
    
    loss_bc_f1 =  Loss_BC_fluid_left + Loss_BC_fluid_right + Loss_BC_inlet_u + Loss_BC_inlet_v + Loss_BC_outlet_v + Loss_BC_outlet_p
    
    Loss_BC_fluid_left = Loss_func(Loss_BC_F2[0].float(), torch.zeros_like(Loss_BC_F2[0]).to(device))
    Loss_BC_fluid_right = Loss_func(Loss_BC_F2[1].float(), torch.zeros_like(Loss_BC_F2[1]).to(device))
    Loss_BC_inlet_u = Loss_func(Loss_BC_F2[2].float(), torch.zeros_like(Loss_BC_F2[2]).to(device))
    Loss_BC_inlet_v = Loss_func(Loss_BC_F2[3].float(), torch.zeros_like(Loss_BC_F2[3]).to(device))
    Loss_BC_outlet_v = Loss_func(Loss_BC_F2[4].float(), torch.zeros_like(Loss_BC_F2[4]).to(device))
    Loss_BC_outlet_p = Loss_func(Loss_BC_F2[5].float(), torch.zeros_like(Loss_BC_F2[5]).to(device))
    
    loss_bc_f2 =  Loss_BC_fluid_left + Loss_BC_fluid_right + Loss_BC_inlet_u + Loss_BC_inlet_v + Loss_BC_outlet_v + Loss_BC_outlet_p
    
    loss1 = loss_pde_f1 + loss_bc_f1 + loss_pde_f2 + loss_bc_f2
    
    optimizer1.zero_grad()
    loss1.backward()
    optimizer1.step()
    
    # Loss about fluid heat exchange
    Loss_PDE_H_F1 = PDE_H_F1(model1, model2, collocation_points)
    Loss_BC_T1 = BC_T1(model2, collocation_points)
    Loss_PDE_H_F2 = PDE_H_F2(model1, model2, collocation_points)
    Loss_BC_T2 = BC_T2(model2, collocation_points)
    Loss_PDE_S = PDE_S(model2, collocation_points)

    Loss_PDE_H_F1_convection = Loss_func(Loss_PDE_H_F1[0].float(), torch.zeros_like(Loss_PDE_H_F1[0]).to(device)) 
    Loss_PDE_H_F2_convection = Loss_func(Loss_PDE_H_F2[0].float(), torch.zeros_like(Loss_PDE_H_F2[0]).to(device)) 
    Loss_PDF_S_conduction = Loss_func(Loss_PDE_S[0].float(), torch.zeros_like(Loss_PDE_S[0]).to(device))

    Loss_BC1_fluid_left_T = Loss_func(Loss_BC_T1[0].float(), torch.zeros_like(Loss_BC_T1[0]).to(device))
    Loss_BC1_outlet_dT_x = Loss_func(Loss_BC_T1[1].float(), torch.zeros_like(Loss_BC_T1[1]).to(device))
    Loss_BC1_inlet_T = Loss_func(Loss_BC_T1[2].float(), torch.zeros_like(Loss_BC_T1[2]).to(device))

    loss_bc_t1 = Loss_BC1_fluid_left_T + Loss_BC1_outlet_dT_x + Loss_BC1_inlet_T
    
    Loss_BC2_fluid_left_T = Loss_func(Loss_BC_T2[0].float(), torch.zeros_like(Loss_BC_T2[0]).to(device))
    Loss_BC2_outlet_dT_x = Loss_func(Loss_BC_T2[1].float(), torch.zeros_like(Loss_BC_T2[1]).to(device))
    Loss_BC2_inlet_T = Loss_func(Loss_BC_T2[2].float(), torch.zeros_like(Loss_BC_T2[2]).to(device))

    loss_bc_t2 = Loss_BC2_fluid_left_T + Loss_BC2_outlet_dT_x + Loss_BC2_inlet_T

    loss2 = Loss_PDE_H_F1_convection + loss_bc_t1 + Loss_PDE_H_F2_convection + loss_bc_t2 + 25 * (Loss_PDF_S_conduction)

    optimizer2.zero_grad()
    loss2.backward()
    optimizer2.step()

    loss_total = loss1.item() + loss2.item()
    loss_model1 = loss1.item()
    loss_model2 = loss2.item()

    if loss_total < max_loss:
        torch.save(model1, './weights/he_pinn_MPINN/MPINN_model1.pt')
        torch.save(model2, './weights/he_pinn_MPINN/MPINN_model2.pt')
        max_loss = loss_total

    if epoch % 1000 == 0:
        with torch.no_grad():
            model1_test = torch.load('./weights/he_pinn_MPINN/MPINN_model1.pt')
            model2_test = torch.load('./weights/he_pinn_MPINN/MPINN_model2.pt')
            print('Epoch: {} Loss: {:.4f} Loss1: {:.4f} Loss2: {:.4f}'.format(epoch, loss_total, loss_model1, loss_model2))
            PLOT(model1_test, model2_test, cor_fluid1, cor_fluid2, cor_solid, CFD_results_fluid1, CFD_results_fluid2, CFD_results_solid, epoch, history=False)
    epoch += 1